In [1]:
import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import random
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
    
sys.path.append('xgboost/wrapper/')
import xgboost
from xgboost import XGBClassifier
    
#input data
train_df=pd.read_json('data/train.json')
# build_df=pd.read_csv("data/train_new.csv")
test_df=pd.read_json('data/test.json')
# train_df["building_id"] = build_df["building_id"]

In [2]:
def feature_engineering(df):
    
    df["created"] = pd.to_datetime(df["created"])
    df["days_since"] = (df["created"].max() - df["created"])
    df["days_since"] = (df["days_since"] / np.timedelta64(1, 'D')).astype(int)
    df["created_month"] = df["created"].dt.month
    df["created_day"] = df["created"].dt.day
    df["created_hour"] = df["created"].dt.hour
    df["is_night"] = 1*(df["created_hour"] <= 7)
    df["created_weekday"] = df["created"].dt.weekday
    df["is_weekend"] = 1*(df["created_weekday"] > 5)
    
    df["total_rooms"] = df["bedrooms"] + df["bathrooms"]
    df["diff_rooms"] = df['bedrooms'] - df['bathrooms']
    df['price_per_bed'] = df['price']/(df['bedrooms']+1)
    df['price_per_bath'] = df['price']/(df['bathrooms']+1)
    df["price_per_room"] = df['price']/(df['total_rooms']+1)
    
    df['description'] = df['description'].str.replace('<[^<>]+>', ' ')
    df['description'] = df['description'].str.replace('[0-9]+', 'num')
    df['description'] = df['description'].str.lower().replace('[^a-zA-Z0-9]', ' ')
    
    df["num_features"] = df["features"].apply(len)
    df["num_description_words"] = df["description"].apply(lambda x: len(x.split(" ")))

    df["num_photos"] = df["photos"].apply(len)
    df["num_photos_per_room"] = df['num_photos'] / df['total_rooms']
    
    df["pos"] = df.longitude.round(3).astype(str) + '_' + df.latitude.round(3).astype(str)
    vals = df['pos'].value_counts()
    dvals = vals.to_dict()
    df["density"] = df['pos'].apply(lambda x: dvals.get(x, vals.min()))
            
    df["logprice"] = np.log(df["price"])

    num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_features", 
    "num_description_words", "days_since", "created_month", "created_day", "created_hour", "created_weekday", 
    "is_night", "is_weekend", "num_photos", "price_per_bed", "price_per_bath", "total_rooms","diff_rooms",
    "price_per_room", "num_photos_per_room", "logprice", "density", "description", "features"]
    X = df[num_feats]
    return X

train = feature_engineering(df=train_df)
test = feature_engineering(df=test_df)

In [3]:
index=list(range(train_df.shape[0]))
random.shuffle(index)
a=[np.nan]*len(train_df)
b=[np.nan]*len(train_df)
c=[np.nan]*len(train_df)

for i in range(5):
    building_level={}
    for j in train_df['manager_id'].values:
        building_level[j]=[0,0,0]
    test_index=index[int((i*train_df.shape[0])/5):int(((i+1)*train_df.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    for j in train_index:
        temp=train_df.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
    for j in test_index:
        temp=train_df.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
train_df['manager_level_low']=a
train_df['manager_level_medium']=b
train_df['manager_level_high']=c

train['manager_level_low'] = train_df['manager_level_low']
train['manager_level_medium'] = train_df['manager_level_medium']
train['manager_level_high'] = train_df['manager_level_high']

a=[]
b=[]
c=[]
building_level={}
for j in train_df['manager_id'].values:
    building_level[j]=[0,0,0]
for j in range(train_df.shape[0]):
    temp=train_df.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in test_df['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))
test_df['manager_level_low']=a
test_df['manager_level_medium']=b
test_df['manager_level_high']=c

test['manager_level_low'] = test_df['manager_level_low']
test['manager_level_medium'] = test_df['manager_level_medium']
test['manager_level_high'] = test_df['manager_level_high']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [4]:

categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            
train['display_address'] = train_df['display_address']         
train['manager_id'] = train_df['manager_id']
train['building_id'] = train_df['building_id']
train['street_address'] = train_df['street_address']

test['display_address'] = test_df['display_address']         
test['manager_id'] = test_df['manager_id']
test['building_id'] = test_df['building_id']
test['street_address'] = test_df['street_address']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [5]:
from math import sin, cos, sqrt, atan2, radians

def get_dist(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

def get_dist_features(df):
    df["cp_dist"] = list(map(lambda lat2, lon2: get_dist(40.783661, -73.96536827, lat2, lon2), 
    df["latitude"], df["longitude"]))
    df["jfk_dist"] = list(map(lambda lat2, lon2: get_dist(40.64912697, -73.78692627, lat2, lon2), 
    df["latitude"], df["longitude"]))
    df["wsp_dist"] = list(map(lambda lat2, lon2: get_dist(40.73083612, -73.99749041, lat2, lon2), 
    df["latitude"], df["longitude"]))
    df["fd_dist"] = list(map(lambda lat2, lon2: get_dist(40.705628, -74.010278, lat2, lon2), 
    df["latitude"], df["longitude"]))
    df["ca_dist"] = list(map(lambda lat2, lon2: get_dist(40.849209,-73.888508, lat2, lon2), 
    df["latitude"], df["longitude"]))
    df["61_dist"] = list(map(lambda lat2, lon2: get_dist(40.747844,-73.901731, lat2, lon2), 
    df["latitude"], df["longitude"]))
    df["aa_dist"] = list(map(lambda lat2, lon2: get_dist(40.678722,-73.951174, lat2, lon2), 
    df["latitude"], df["longitude"]))
    df["la_dist"] = list(map(lambda lat2, lon2: get_dist(40.688788,-73.870111, lat2, lon2), 
    df["latitude"], df["longitude"]))
    df["ja_dist"] = list(map(lambda lat2, lon2: get_dist(40.624861,-73.967846, lat2, lon2), 
    df["latitude"], df["longitude"]))
    

    features = ["cp_dist", "jfk_dist", "wsp_dist", "fd_dist", "ca_dist", "61_dist", "aa_dist", "la_dist", "ja_dist"]
    
    return df[features]

dist_features = get_dist_features(train_df)
train = pd.concat([train, dist_features], axis=1)

dist_features_test = get_dist_features(test_df)
test = pd.concat([test, dist_features_test], axis=1)

# test["cp_dist"] = test_df["cp_dist"]
# test["jfk_dist"] = test_df["jfk_dist"]
# test["wsp_dist"] = test_df["wsp_dist"]
# test["fd_dist"] = test_df["fd_dist"]
# test["ca_dist"] = test_df["ca_dist"]
# test["61_dist"] = test_df["61_dist"]
# test["aa_dist"] = test_df["aa_dist"]
# test["la_dist"] = test_df["la_dist"]
# test["ja_dist"] = test_df["ja_dist"]

In [6]:
def get_train_clusters (train_df):
    latlong = train_df.copy () [['latitude', 'longitude']]
    n_cluster = 15
    clf = KMeans(n_clusters=n_cluster)
    clf.fit(latlong)
    train_df['cluster_id'] = clf.labels_
    
    return train_df['cluster_id'], clf

train_clusters, clf = get_train_clusters (train_df)
print(train_clusters.head())

def get_test_clusters (test_df, clf):
    latlong = test_df.copy () [['latitude', 'longitude']]
    testLabels = [clf.predict([x]) for x in test_df.copy()[['latitude','longitude']].values]
    test_df["cluster_id"] = np.asarray(testLabels)
    
    return test_df['cluster_id']

test_clusters = get_test_clusters(test_df, clf)
print(test_clusters.head())

10         7
100004     0
100007    10
100014     0
100026     5
Name: cluster_id, dtype: int32
10000     5
100013    5
100016    5
100020    0
100099    0
Name: cluster_id, dtype: int32


In [7]:
train = pd.concat([train, train_clusters], axis=1)
test = pd.concat([test, test_clusters], axis=1)

In [8]:
def get_count_features(df):
    m_count = df.groupby(["manager_id"]).count().iloc[:, 1].to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "num_listing_manager"] = m_count[df.loc[df.index[i], "manager_id"]]
    
    b_count = df.groupby(["building_id"]).count().iloc[:, 1].to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "num_listing_build"] = b_count[df.loc[df.index[i], "building_id"]]
        
    sa_count = df.groupby(["street_address"]).count().iloc[:, 1].to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "num_listing_sa"] = sa_count[df.loc[df.index[i], "street_address"]]
        
    da_count = df.groupby(["display_address"]).count().iloc[:, 1].to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "num_listing_da"] = da_count[df.loc[df.index[i], "display_address"]]
    
    p_med_man = df.groupby(["manager_id"]).price.median().to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "median_price_g_manager"] = p_med_man[df.loc[df.index[i], "manager_id"]]
    
    p_med_bui = df.groupby(["building_id"]).price.median().to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "median_price_g_build"] = p_med_bui[df.loc[df.index[i], "building_id"]]
        
    p_std_man = df.groupby(["manager_id"]).price.std().to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "std_price_g_manager"] = p_std_man[df.loc[df.index[i], "manager_id"]]
    
    p_std_bui = df.groupby(["building_id"]).price.std().to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "std_price_g_build"] = p_std_bui[df.loc[df.index[i], "building_id"]]
        
    p_std_clus = df.groupby(["cluster_id"]).price.std().to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "std_price_g_clus"] = p_std_clus[df.loc[df.index[i], "cluster_id"]]
        
    p_med_clus = df.groupby(["cluster_id"]).price.median().to_dict()
    for i in range(df.shape[0]):
        df.loc[df.index[i], "med_price_g_clus"] = p_med_clus[df.loc[df.index[i], "cluster_id"]]
        
    features=["num_listing_manager", "num_listing_build", "num_listing_sa", "num_listing_da", 
              "median_price_g_manager", "median_price_g_build", "std_price_g_manager", "std_price_g_build",
             "med_price_g_clus", "std_price_g_clus"]
    
    return df[features]

df_count = get_count_features(df=train_df)
df_test_count = get_count_features(df=test_df)

In [85]:
train = pd.concat([train, df_count], axis=1)
test = pd.concat([test, df_test_count], axis=1)

In [86]:
train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
print(train_df["features"].head())

10                                                         
100004    Laundry_In_Building Dishwasher Hardwood_Floors...
100007                               Hardwood_Floors No_Fee
100014                                                     
100026    Cats_Allowed Dogs_Allowed Elevator Laundry_In_...
Name: features, dtype: object


In [87]:
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

In [43]:
# tfidf_d = TfidfVectorizer(stop_words='english', max_features=100, ngram_range=(1,2))
# tr_desc_sparse = tfidf_d.fit_transform(train_df["description"])
# te_desc_sparse = tfidf_d.transform(test_df["description"])

In [44]:
# xtrain = sparse.hstack([train_df[features_to_use], tr_desc_sparse]).tocsr()
# xtest = sparse.hstack([test_df[features_to_use], te_desc_sparse]).tocsr()

In [88]:
# features_to_use = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_features", 
#     "num_description_words", "days_since", "created_month", "created_day", "created_hour", "created_weekday", 
#     "is_night", "is_weekend", "num_photos", "price_per_bed", "price_per_bath", "total_rooms","diff_rooms",
#     "price_per_room", "num_photos_per_room", "logprice",
#     "density", "num_listing_manager", "num_listing_build", 
#     "num_listing_sa", "num_listing_da", "median_price_g_manager", "median_price_g_build", "std_price_g_manager", 
#     "std_price_g_build", "manager_level_low", "manager_level_medium", "manager_level_high", 
#     "display_address", "manager_id", "building_id", "street_address", 
#     "cp_dist", "jfk_dist", "wsp_dist", "fd_dist", "ca_dist", "61_dist", "aa_dist", "la_dist", "ja_dist", "cluster_id",
#     "median_price_g_clus", "std_price_g_clus"]

features_to_use = ["bathrooms", "bedrooms", "price", "num_features", 
    "num_description_words", "days_since", "created_month", "created_day", "created_hour", "created_weekday", 
    "is_night", "is_weekend", "num_photos", "price_per_bed", "price_per_bath", "total_rooms","diff_rooms",
    "price_per_room", "num_photos_per_room", "logprice",
    "density", "num_listing_manager", "num_listing_build", 
    "num_listing_sa", "num_listing_da", "median_price_g_manager", "median_price_g_build", "std_price_g_manager", 
    "std_price_g_build", "manager_level_low", "manager_level_medium", "manager_level_high", 
    "display_address", "manager_id", "building_id", "street_address", 
    "cp_dist", "jfk_dist", "wsp_dist", "fd_dist", "ca_dist", "61_dist", "aa_dist", "la_dist", "ja_dist", "cluster_id",
    "med_price_g_clus", "std_price_g_clus"]

# features_to_use = ["bathrooms", "bedrooms", "price", "num_features", 
#     "num_description_words", "created_month", "created_day", "created_hour", "created_weekday", 
#     "is_night", "is_weekend", "num_photos", "price_per_bed", "price_per_bath", "total_rooms","diff_rooms",
#     "price_per_room", "num_photos_per_room", "logprice",
#     "density", "num_listing_manager", 
#     "num_listing_sa", "num_listing_da", "median_price_g_manager", "median_price_g_build", "std_price_g_manager", 
#     "std_price_g_build", "manager_level_low", "manager_level_medium", "manager_level_high", 
#     "display_address", "manager_id", "building_id", "street_address", 
#     "cp_dist", "jfk_dist", "wsp_dist", "fd_dist", "ca_dist", "61_dist", "aa_dist", "la_dist", "ja_dist"]

print(len(features_to_use))


48


In [89]:
train_X = sparse.hstack([train_df[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse]).tocsr()

In [90]:
target_num_map = {'high':0, 'low':1, 'medium':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

In [97]:
# def runXGB1(X_train, y_train, X_test, y_test):
#     model = XGBClassifier(max_depth=6, learning_rate=0.03, n_estimators=700, min_child_weight=3, 
#                           subsample=0.7, colsample_bytree=0.7, n_jobs=-1, silent=0, objective='multi:softprob')
    
#     model.fit(X_train, y_train)
#     y_train_pred = model.predict_proba(X_train)
    
#     print("Log loss on train: %f"%(log_loss(y_train, y_train_pred)))
    
#     y_test_pred = model.predict_proba(X_test)
    
#     print("Log loss on train: %f"%(log_loss(y_test, y_test_pred)))
    
#     return y_test_pred, model
    
def runXGB2(X_train, y_train, X_test):
    model = XGBClassifier(max_depth=7, learning_rate=0.03, n_estimators=800, gamma=3, min_child_weight=3, 
                          subsample=0.6, colsample_bytree=0.6, colsample_bylevel=0.6, reg_alpha=1.5,
                          reg_lambda=2.5, n_jobs=-1, silent=0, objective='multi:softprob')
    
    model.fit(X_train, y_train)
    y_train_pred = model.predict_proba(X_train)
    
    print("Log loss on train: %f"%(log_loss(y_train, y_train_pred)))
    
    y_test_pred = model.predict_proba(X_test)
    
#     print("Log loss on train: %f"%(log_loss(y_test, y_test_pred)))
    
    return y_test_pred, model

# def runRF(X_train, y_train, X_test, y_test):
#     model = RandomForestClassifier(n_estimators=500, max_depth=20, max_features='auto'
#         , min_samples_leaf=70, min_samples_split=50, n_jobs=-1)
#     print("\nRandom Forest: Training...\n")
#     model.fit(X_train, y_train)

#     y_train_pred = model.predict_proba(X_train)
#     print("Random Forest: Log loss on training set: %f" %log_loss(y_train, y_train_pred))

#     y_test_pred = model.predict_proba(X_test)
        
#     print("\nRandom Forest: Log loss on validation set: %f" %log_loss(y_test, y_test_pred))


In [98]:
#Use voting classifier
# pred1, model1 = runXGB1(train_X, train_y, test_X, num_rounds=1500)
pred2, model2 = runXGB2(train_X, train_y, test_X)

[23:14:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 2 pruned nodes, max_depth=4
[23:14:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:14:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[23:14:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:14:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:08] /work

[23:14:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:14:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:14:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:14:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:14:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:14:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[23:14:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:11] /work

[23:14:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:14:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:14:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:14:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:14:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:14:15] /work

[23:14:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:14:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:14:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:14:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=4
[23:14:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 4 pruned nodes, max_depth=4
[23:14:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:14:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[23:14:18] /work

[23:14:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[23:14:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=4
[23:14:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:14:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 4 pruned nodes, max_depth=4
[23:14:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 4 pruned nodes, max_depth=4
[23:14:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:14:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=4
[23:14:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:14:21] /work

...

[23:15:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 4 pruned nodes, max_depth=4
[23:15:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:15:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 6 pruned nodes, max_depth=4
[23:15:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[23:15:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:15:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[23:15:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[23:15:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 6 pruned nodes, max_depth=4
[23:15:29] /work

[23:15:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=4
[23:15:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[23:15:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:15:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[23:15:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:15:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 8 pruned nodes, max_depth=4
[23:15:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=4
[23:15:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:15:32] /work

[23:15:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[23:15:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 6 pruned nodes, max_depth=4
[23:15:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 4 pruned nodes, max_depth=4
[23:15:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[23:15:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[23:15:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:15:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[23:15:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:15:34] /work

[23:15:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:15:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 8 pruned nodes, max_depth=4
[23:15:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:15:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=4
[23:15:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 4 pruned nodes, max_depth=4
[23:15:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 4 pruned nodes, max_depth=4
[23:15:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=4
[23:15:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[23:15:37] /works

[23:15:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[23:15:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=4
[23:15:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[23:15:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[23:15:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[23:15:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:15:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[23:15:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[23:15:39] /work

In [ ]:
# model = VotingClassifier(estimators=[('xgb1', model1), ('xgb2', model2)], voting='soft', n_jobs=-1)
# model.fit(train_X,train_y)
# y_pred = model.predict(test_X)

In [75]:
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_X.shape[0])):
        dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB2(dev_X, dev_y, val_X, val_y)
        cv_scores.append(log_loss(val_y, preds))
        print(cv_scores)
        break

[03:20:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[03:20:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[03:20:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 4 pruned nodes, max_depth=4
[03:20:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 4 pruned nodes, max_depth=4
[03:20:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[03:20:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[03:20:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=4
[03:20:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[03:20:57] /work

[03:20:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[03:20:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=4
[03:20:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 10 pruned nodes, max_depth=4
[03:21:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 2 pruned nodes, max_depth=4
[03:21:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 14 pruned nodes, max_depth=4
[03:21:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=4
[03:21:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[03:21:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[03:21:00] /wo

[03:21:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=4
[03:21:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[03:21:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 10 pruned nodes, max_depth=4
[03:21:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=4
[03:21:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[03:21:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=4
[03:21:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=4
[03:21:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 8 pruned nodes, max_depth=4
[03:21:03] /w

[03:21:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 10 pruned nodes, max_depth=4
[03:21:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 12 pruned nodes, max_depth=4
[03:21:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 8 pruned nodes, max_depth=4
[03:21:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 10 pruned nodes, max_depth=4
[03:21:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 10 pruned nodes, max_depth=4
[03:21:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=4
[03:21:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[03:21:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=4
[03:21:05] /

[03:21:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 10 pruned nodes, max_depth=4
[03:21:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=4
[03:21:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 10 pruned nodes, max_depth=4
[03:21:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 4 pruned nodes, max_depth=4
[03:21:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 12 pruned nodes, max_depth=4
[03:21:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 10 pruned nodes, max_depth=4
[03:21:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=4
[03:21:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[03:21:06] 

...

[03:21:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[03:21:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 14 pruned nodes, max_depth=4
[03:21:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[03:21:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[03:21:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 16 pruned nodes, max_depth=4
[03:21:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[03:21:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[03:21:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 18 pruned nodes, max_depth=4
[03:21:53] /wo

[03:21:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[03:21:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 24 pruned nodes, max_depth=3
[03:21:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[03:21:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 24 pruned nodes, max_depth=2
[03:21:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 26 pruned nodes, max_depth=2
[03:21:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[03:21:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 26 pruned nodes, max_depth=0
[03:21:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes, max_depth=3
[03:21:54] /wor

[03:21:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[03:21:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[03:21:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 26 pruned nodes, max_depth=0
[03:21:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 12 pruned nodes, max_depth=4
[03:21:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 26 pruned nodes, max_depth=0
[03:21:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[03:21:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 24 pruned nodes, max_depth=3
[03:21:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 24 pruned nodes, max_depth=3
[03:21:56] /wor

[03:21:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 24 pruned nodes, max_depth=3
[03:21:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[03:21:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[03:21:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[03:21:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[03:21:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 10 pruned nodes, max_depth=4
[03:21:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[03:21:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 18 pruned nodes, max_depth=0
[03:21:58] /wor

[03:22:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[03:22:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 18 pruned nodes, max_depth=4
[03:22:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[03:22:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 26 pruned nodes, max_depth=0
[03:22:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 20 pruned nodes, max_depth=4
[03:22:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[03:22:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 20 pruned nodes, max_depth=3
[03:22:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 28 pruned nodes, max_depth=0
[03:22:00] /wo

In [26]:
# preds, model = runXGB2(train_X, train_y, test_X, num_rounds=1500)

In [99]:
def pred_to_csv(model_name, y_test):
    y_pred = pd.DataFrame()
    y_pred['listing_id'] = test_df['listing_id']
    for i in range(len(list(y_test))):
        [y_pred.loc[y_pred.index[i], 'high'], y_pred.loc[y_pred.index[i], 'low'], y_pred.loc[y_pred.index[i], 'medium']] = list(y_test[i])

    print(y_pred.head())

    y_pred.to_csv("results/pred_"+ model_name +".csv", index=False)

    print("\n\nDone! CSV for "+model_name+"'s predictions created!\n")
    
pred_to_csv("xgb_30", pred2)

        listing_id      high       low    medium
10000         5529  0.001330  0.984535  0.014136
100013       14597  0.002109  0.980763  0.017128
100016       43396  0.004018  0.960151  0.035832
100020       25538  0.008937  0.912317  0.078746
100099       26644  0.003718  0.874209  0.122073


Done! CSV for xgb_30's predictions created!

